In [1]:
import gurobipy as gp
import numpy as np
import math
from numpy import random

In [19]:
m = 11
max_val = 100
U1 = random.randint(max_val, size=(m,m,m))
U2 = random.randint(max_val, size=(m,m,m))
U3 = random.randint(max_val, size=(m,m,m))

#Defining Dictionaries for various variables
y13 = {}
y23 = {}
z = {}
x1 = {}
x2 = {}
x3 = {}
s1 = {}
s2 = {}


In [20]:
M = gp.Model()

#Adding all the variables

for i in range(m):
    s1[i] = M.addVar(vtype = gp.GRB.BINARY, name = "s1"+str(i))
    s2[i] = M.addVar(vtype = gp.GRB.BINARY, name = "s2"+str(i))
    x1[i] = M.addVar(lb = 0, vtype = gp.GRB.CONTINUOUS, name = "x1"+str(i))
    for j in range(m):
        y13[i,j] = M.addVar(lb = 0, ub = 1, vtype = gp.GRB.CONTINUOUS, name = "y13"+str(i)+'_'+str(j))
        x2[j] = M.addVar(lb = 0, vtype = gp.GRB.CONTINUOUS, name = "x2"+str(j))
        for k in range(m):
            y23[j,k] = M.addVar(lb = 0, ub = 1, vtype = gp.GRB.CONTINUOUS, name = "y23"+str(j)+'_'+str(k))
            z[i,j,k] = M.addVar(lb = 0, ub = 1, vtype = gp.GRB.CONTINUOUS, name = "z"+str(i)+'_'+str(j)+'_'+str(k))
            x3[k] = M.addVar(lb = 0, vtype = gp.GRB.CONTINUOUS, name = "x3"+str(k))
            
v1 = M.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype = gp.GRB.CONTINUOUS)
v2 = M.addVar(lb = -gp.GRB.INFINITY, ub = gp.GRB.INFINITY, vtype = gp.GRB.CONTINUOUS)

BigM = 10000000000

In [21]:
#Objective Function

M.setObjective(gp.quicksum(U3[i,j,k]*z[i,j,k] for i in range(m) for j in range(m) for k in range(m)), sense = gp.GRB.MAXIMIZE)

In [22]:
#Constraints

M.addConstr(gp.quicksum(y13[i,k] for i in range(m) for j in range(m))==1)
M.addConstr(gp.quicksum(y23[j,k] for j in range(m) for k in range(m))==1)
M.addConstr(gp.quicksum(z[i,j,k] for i in range(m) for j in range(m) for k in range(m))==1)

M.addConstr(gp.quicksum(x1[i] for i in range(m))==1)
M.addConstr(gp.quicksum(x2[j] for j in range(m))==1)
M.addConstr(gp.quicksum(x3[k] for k in range(m))==1)

for i in range(m):
    M.addConstr(v1 - gp.quicksum(U1[i,j,k]*y23[j,k] for j in range(m) for k in range(m))<=BigM*s1[i])
    M.addConstr(x1[i]<=1-s1[i])
    M.addConstr(v1 >= gp.quicksum(U1[i,j,k]*y23[j,k] for j in range(m) for k in range(m)))
    for k in range(m):
        M.addConstr(y13[i,k]==x1[i]*x3[k])
        
for j in range(m):
    M.addConstr(v2 - gp.quicksum(U2[i,j,k]*y13[i,k] for i in range(m) for k in range(m))<=BigM*s1[i])
    M.addConstr(x2[j]<=1-s2[j])
    M.addConstr(v2 >= gp.quicksum(U2[i,j,k]*y13[i,k] for i in range(m) for k in range(m)))
    for k in range(m):
        M.addConstr(y23[j,k]==x2[j]*x3[k])

M.addConstrs(z[i,j,k]==x1[i]*y23[j,k] for i in range(m) for j in range(m) for k in range(m))
M.update()
    

In [23]:
M.params.NonConvex = 2
M.params.OutputFlag=1
M.optimize()


Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 72 rows, 4270 columns and 6870 nonzeros
Model fingerprint: 0x52db9edc
Model has 1573 quadratic constraints
Variable types: 4248 continuous, 22 integer (22 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+10]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 11 rows and 2651 columns
Presolve time: 0.04s
Presolved: 6353 rows, 1619 columns, 19408 nonzeros
Presolved model has 1573 bilinear constraint(s)
Variable types: 1608 continuous

  7665  1195     cutoff   23        94.81818   97.67314  3.01%   150  307s
  7874  1208   95.61527   23  250   94.81818   97.67314  3.01%   150  311s
  8085  1251   97.18899   22  113   94.81818   97.67314  3.01%   151  316s
  8324  1271   97.67314   21  166   94.81818   97.67314  3.01%   150  321s
  8492  1287     cutoff   23        94.81818   97.67314  3.01%   152  327s
  8754  1292   96.81624   24  159   94.81818   97.67314  3.01%   151  332s
  8958  1311     cutoff   24        94.81818   97.67314  3.01%   152  338s
  9155  1319   95.32527   22  152   94.81818   97.67314  3.01%   153  344s
  9373  1328     cutoff   24        94.81818   97.67314  3.01%   154  349s
  9630  1329   95.35871   25  150   94.81818   97.67314  3.01%   155  357s
  9878  1320     cutoff   24        94.81818   97.67314  3.01%   155  364s
 10139  1334   96.01238   24   96   94.81818   97.67314  3.01%   155  372s
 10454  1348   97.22968   25  156   94.81818   97.67314  3.01%   155  379s
 10773  1337   95.52513  

In [18]:
print(M.runtime, M.objVal)

0.14148712158203125 93.99999999999999
